# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f85bc956820>
├── 'a' --> tensor([[-1.1338,  0.9034,  1.9563],
│                   [-0.2072, -0.3906, -0.0051]])
└── 'x' --> <FastTreeValue 0x7f85bc9569a0>
    └── 'c' --> tensor([[-2.1356, -1.2087, -2.2359,  0.8187],
                        [-0.0879, -1.0323, -1.0386, -0.5144],
                        [-0.8357,  0.4749, -1.2028, -0.8404]])

In [4]:
t.a

tensor([[-1.1338,  0.9034,  1.9563],
        [-0.2072, -0.3906, -0.0051]])

In [5]:
%timeit t.a

90.1 ns ± 1.63 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f85bc956820>
├── 'a' --> tensor([[-0.0858,  0.0588, -0.2778],
│                   [ 0.6746,  1.4744,  0.2757]])
└── 'x' --> <FastTreeValue 0x7f85bc9569a0>
    └── 'c' --> tensor([[-2.1356, -1.2087, -2.2359,  0.8187],
                        [-0.0879, -1.0323, -1.0386, -0.5144],
                        [-0.8357,  0.4749, -1.2028, -0.8404]])

In [7]:
%timeit t.a = new_value

87.8 ns ± 1.62 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.1338,  0.9034,  1.9563],
               [-0.2072, -0.3906, -0.0051]]),
    x: Batch(
           c: tensor([[-2.1356, -1.2087, -2.2359,  0.8187],
                      [-0.0879, -1.0323, -1.0386, -0.5144],
                      [-0.8357,  0.4749, -1.2028, -0.8404]]),
       ),
)

In [10]:
b.a

tensor([[-1.1338,  0.9034,  1.9563],
        [-0.2072, -0.3906, -0.0051]])

In [11]:
%timeit b.a

74.8 ns ± 2.36 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.7424,  1.9736, -1.3229],
               [ 1.2073, -1.4259, -0.8783]]),
    x: Batch(
           c: tensor([[-2.1356, -1.2087, -2.2359,  0.8187],
                      [-0.0879, -1.0323, -1.0386, -0.5144],
                      [-0.8357,  0.4749, -1.2028, -0.8404]]),
       ),
)

In [13]:
%timeit b.a = new_value

687 ns ± 22.1 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.07 µs ± 13.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

14.4 µs ± 194 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

313 µs ± 11.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

304 µs ± 22.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f85be9c7c10>
├── 'a' --> tensor([[[-1.1338,  0.9034,  1.9563],
│                    [-0.2072, -0.3906, -0.0051]],
│           
│                   [[-1.1338,  0.9034,  1.9563],
│                    [-0.2072, -0.3906, -0.0051]],
│           
│                   [[-1.1338,  0.9034,  1.9563],
│                    [-0.2072, -0.3906, -0.0051]],
│           
│                   [[-1.1338,  0.9034,  1.9563],
│                    [-0.2072, -0.3906, -0.0051]],
│           
│                   [[-1.1338,  0.9034,  1.9563],
│                    [-0.2072, -0.3906, -0.0051]],
│           
│                   [[-1.1338,  0.9034,  1.9563],
│                    [-0.2072, -0.3906, -0.0051]],
│           
│                   [[-1.1338,  0.9034,  1.9563],
│                    [-0.2072, -0.3906, -0.0051]],
│           
│                   [[-1.1338,  0.9034,  1.9563],
│                    [-0.2072, -0.3906, -0.0051]]])
└── 'x' --> <FastTreeValue 0x7f84e7fa8850>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

51.1 µs ± 878 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f84dfa2b0d0>
├── 'a' --> tensor([[-1.1338,  0.9034,  1.9563],
│                   [-0.2072, -0.3906, -0.0051],
│                   [-1.1338,  0.9034,  1.9563],
│                   [-0.2072, -0.3906, -0.0051],
│                   [-1.1338,  0.9034,  1.9563],
│                   [-0.2072, -0.3906, -0.0051],
│                   [-1.1338,  0.9034,  1.9563],
│                   [-0.2072, -0.3906, -0.0051],
│                   [-1.1338,  0.9034,  1.9563],
│                   [-0.2072, -0.3906, -0.0051],
│                   [-1.1338,  0.9034,  1.9563],
│                   [-0.2072, -0.3906, -0.0051],
│                   [-1.1338,  0.9034,  1.9563],
│                   [-0.2072, -0.3906, -0.0051],
│                   [-1.1338,  0.9034,  1.9563],
│                   [-0.2072, -0.3906, -0.0051]])
└── 'x' --> <FastTreeValue 0x7f84dfa2b5b0>
    └── 'c' --> tensor([[-2.1356, -1.2087, -2.2359,  0.8187],
                        [-0.0879, -1.0323, -1.0386, -0.5144],
                 

In [23]:
%timeit t_cat(trees)

47.7 µs ± 1.05 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

101 µs ± 788 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-2.1356, -1.2087, -2.2359,  0.8187],
                       [-0.0879, -1.0323, -1.0386, -0.5144],
                       [-0.8357,  0.4749, -1.2028, -0.8404]],
              
                      [[-2.1356, -1.2087, -2.2359,  0.8187],
                       [-0.0879, -1.0323, -1.0386, -0.5144],
                       [-0.8357,  0.4749, -1.2028, -0.8404]],
              
                      [[-2.1356, -1.2087, -2.2359,  0.8187],
                       [-0.0879, -1.0323, -1.0386, -0.5144],
                       [-0.8357,  0.4749, -1.2028, -0.8404]],
              
                      [[-2.1356, -1.2087, -2.2359,  0.8187],
                       [-0.0879, -1.0323, -1.0386, -0.5144],
                       [-0.8357,  0.4749, -1.2028, -0.8404]],
              
                      [[-2.1356, -1.2087, -2.2359,  0.8187],
                       [-0.0879, -1.0323, -1.0386, -0.5144],
                       [-0.8357,  0.4749, -1.2028, -0.8404]],

In [26]:
%timeit Batch.stack(batches)

113 µs ± 3.97 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-2.1356, -1.2087, -2.2359,  0.8187],
                      [-0.0879, -1.0323, -1.0386, -0.5144],
                      [-0.8357,  0.4749, -1.2028, -0.8404],
                      [-2.1356, -1.2087, -2.2359,  0.8187],
                      [-0.0879, -1.0323, -1.0386, -0.5144],
                      [-0.8357,  0.4749, -1.2028, -0.8404],
                      [-2.1356, -1.2087, -2.2359,  0.8187],
                      [-0.0879, -1.0323, -1.0386, -0.5144],
                      [-0.8357,  0.4749, -1.2028, -0.8404],
                      [-2.1356, -1.2087, -2.2359,  0.8187],
                      [-0.0879, -1.0323, -1.0386, -0.5144],
                      [-0.8357,  0.4749, -1.2028, -0.8404],
                      [-2.1356, -1.2087, -2.2359,  0.8187],
                      [-0.0879, -1.0323, -1.0386, -0.5144],
                      [-0.8357,  0.4749, -1.2028, -0.8404],
                      [-2.1356, -1.2087, -2.2359,  0.8187],
                   

In [28]:
%timeit Batch.cat(batches)

203 µs ± 9.01 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

767 µs ± 20.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
